In [1]:
import os
import cv2 # OpenCV untuk pemrosesan gambar
import numpy as np # NumPy untuk operasi numerik
from skimage.feature import graycomatrix, graycoprops # Scikit-image untuk GLCM
from sklearn.model_selection import train_test_split # Untuk membagi dataset
from sklearn.preprocessing import LabelEncoder # Untuk mengubah label string menjadi angka
from sklearn.neighbors import KNeighborsClassifier # Classifier K-Nearest Neighbors
from sklearn.metrics import accuracy_score, classification_report # Untuk evaluasi model
from google.colab import drive # Untuk menghubungkan Google Drive
import matplotlib.pyplot as plt # Untuk plotting

In [2]:
try:
    drive.mount('/content/drive')
    dataset_path = '/content/drive/MyDrive/ekspresi_wajah'
    print("Google Drive berhasil di-mount.") # Tambahkan konfirmasi jika berhasil
    print(f"Path dataset diatur ke: {dataset_path}")
except Exception as e:
    print(f"Terjadi error saat me-mount Google Drive atau mengatur path: {e}")
    dataset_path = None # Atau path default jika mount gagal

Mounted at /content/drive
Google Drive berhasil di-mount.
Path dataset diatur ke: /content/drive/MyDrive/ekspresi_wajah


In [15]:
IMAGE_SIZE = (256, 256) # Ukuran gambar sesuai paper [cite: 5]
GLCM_DISTANCES = [1]  # Jarak antar piksel untuk GLCM. Paper tidak menyebutkan secara spesifik, 1 adalah jarak yang umum.
GLCM_ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4] # Sudut 0, 45, 90, 135 derajat [cite: 46]
GLCM_PROPERTIES = ['contrast', 'correlation', 'energy', 'homogeneity'] # Properti GLCM yang diekstrak [cite: 51]

In [21]:
!pip install dlib

In [22]:
import dlib
import requests
import bz2

# Inisialisasi detektor wajah frontal dari dlib
detector = dlib.get_frontal_face_detector()

# Path untuk menyimpan model prediktor landmark
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor_url = f"http://dlib.net/files/{predictor_path}.bz2"

# Unduh model jika belum ada
if not os.path.exists(predictor_path):
    print(f"Mengunduh model prediktor landmark dlib dari {predictor_url}...")
    response = requests.get(predictor_url, stream=True)

    # Dekompresi file .bz2 dan simpan
    with open(predictor_path, "wb") as f_out:
        decompressor = bz2.BZ2Decompressor()
        for chunk in response.iter_content(chunk_size=8192):
            f_out.write(decompressor.decompress(chunk))
    print("Unduhan dan dekompresi selesai.")

# Muat model prediktor landmark
try:
    predictor = dlib.shape_predictor(predictor_path)
    print("Detektor wajah dan prediktor landmark dlib berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat model dlib: {e}")
    print("Pastikan file 'shape_predictor_68_face_landmarks.dat' ada di direktori yang sama.")


Mengunduh model prediktor landmark dlib dari http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2...
Unduhan dan dekompresi selesai.
Detektor wajah dan prediktor landmark dlib berhasil dimuat.


In [23]:
def align_face(image, output_size=IMAGE_SIZE):
    """
    Melakukan penjajaran wajah menggunakan detektor dan landmark dari dlib.
    Menggunakan transformasi affine untuk memetakan wajah ke ukuran output standar.
    """
    # dlib bekerja paling baik pada gambar grayscale
    gray_for_detection = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Deteksi wajah menggunakan dlib
    # Argumen '1' berarti kita melakukan upsample gambar sebanyak 1 kali untuk deteksi lebih baik
    rects = detector(gray_for_detection, 1)

    if len(rects) > 0:
        # Jika ada banyak wajah, ambil yang terbesar
        rect = max(rects, key=lambda r: r.width() * r.height())

        # Prediksi 68 landmark wajah
        shape = predictor(gray_for_detection, rect)

        # Ambil titik landmark untuk mata kiri dan kanan dari dlib
        # Indeks untuk dlib: mata kiri (36-41), mata kanan (42-47)
        left_eye_pts = np.array([(shape.part(i).x, shape.part(i).y) for i in range(36, 42)], np.float32)
        right_eye_pts = np.array([(shape.part(i).x, shape.part(i).y) for i in range(42, 48)], np.float32)

        # Hitung pusat setiap mata
        left_eye_center = np.mean(left_eye_pts, axis=0)
        right_eye_center = np.mean(right_eye_pts, axis=0)

        # Titik sumber: pusat mata kiri, pusat mata kanan, dan hidung (jembatan hidung)
        nose_bridge = shape.part(27) # Jembatan hidung, titik yang stabil
        src_pts = np.float32([left_eye_center, right_eye_center, (nose_bridge.x, nose_bridge.y)])

        # Titik tujuan yang telah ditentukan pada gambar output
        dest_pts = np.float32([
            (output_size[0] * 0.3, output_size[1] * 0.35), # Posisi target mata kiri
            (output_size[0] * 0.7, output_size[1] * 0.35), # Posisi target mata kanan
            (output_size[0] * 0.5, output_size[1] * 0.45)  # Posisi target jembatan hidung
        ])

        # Hitung dan terapkan transformasi affine
        affine_matrix = cv2.getAffineTransform(src_pts, dest_pts)
        aligned_face = cv2.warpAffine(image, affine_matrix, output_size)

        return aligned_face, True

    # Jika tidak ada wajah yang terdeteksi, kembalikan gambar asli
    return image, False

In [24]:
def extract_glcm_features(image):
    # Konversi ke grayscale jika gambar belum grayscale
    if image.ndim > 2 and image.shape[2] > 1:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Pastikan gambar adalah uint8, seperti yang diharapkan oleh graycomatrix
    if gray_image.dtype != np.uint8:
        gray_image = gray_image.astype(np.uint8)


    equalized_image = cv2.equalizeHist(gray_image)

    glcm = graycomatrix(equalized_image, # Gunakan gambar yang sudah di-equalize
                        distances=GLCM_DISTANCES,
                        angles=GLCM_ANGLES,
                        levels=256,  # Mengasumsikan gambar grayscale 8-bit
                        symmetric=True, # Membuat matriks simetris
                        normed=True) # Normalisasi matriks

    features = []
    for prop in GLCM_PROPERTIES:
        prop_values = graycoprops(glcm, prop).ravel()
        features.extend(prop_values)
    return np.array(features)

In [25]:
def load_dataset(dataset_dir):
    images = []
    labels = []
    expression_classes = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])

    if not expression_classes:
        print(f"Tidak ada subdirektori yang ditemukan di {dataset_dir}.")
        return None, None, None

    print(f"Kelas ekspresi yang ditemukan: {expression_classes}")

    alignment_failures = 0
    total_images = 0

    for expression_label in expression_classes:
        expression_path = os.path.join(dataset_dir, expression_label)
        if os.path.isdir(expression_path):
            for image_name in os.listdir(expression_path):
                image_path = os.path.join(expression_path, image_name)
                total_images += 1
                try:
                    img = cv2.imread(image_path)
                    if img is None:
                        print(f"Peringatan: Tidak dapat membaca gambar {image_path}. Melewati.")
                        continue

                    # >>> PERUBAHAN DI SINI: Terapkan Median Filter <<<
                    # Kernel size 3 atau 5 biasanya bagus. Harus ganjil.
                    img_filtered = cv2.medianBlur(img, 3)

                    # Lanjutkan dengan gambar yang sudah difilter
                    processed_img, status = align_face(img_filtered)

                    if not status:
                        alignment_failures += 1
                        # Jika penjajaran gagal, gambar asli (yang sudah difilter) perlu di-resize
                        processed_img = cv2.resize(img_filtered, IMAGE_SIZE)

                    images.append(processed_img)
                    labels.append(expression_label)
                except Exception as e:
                    print(f"Error Kritis saat memproses gambar {image_path}: {e}. Gambar ini dilewati.")

    if not images:
        print("Tidak ada gambar yang dimuat.")
        return None, None, None

    print(f"\nProses pemuatan selesai.")
    print(f"Total gambar yang diproses: {total_images}")
    print(f"Gambar yang berhasil dijajarkan (aligned) dengan dlib: {total_images - alignment_failures}")
    print(f"Gambar yang gagal dijajarkan (menggunakan gambar asli): {alignment_failures}")

    return images, labels, expression_classes

In [26]:
images_data, labels_data, class_names = load_dataset(dataset_path)

if images_data is None or not images_data:
    print("Pemuatan dataset gagal atau dataset kosong. Program berhenti.")
else:
    print(f"Berhasil memuat {len(images_data)} gambar.")

    print("Mengekstrak fitur GLCM...\n") # Tambah baris baru untuk output yang lebih rapi
    features_list = []
    for i, img in enumerate(images_data):
        # Hapus baris ini untuk mengurangi output verbose, karena proses GLCM bisa panjang
        # print(f"Memproses gambar {i+1}/{len(images_data)}...")
        try:
            glcm_feats = extract_glcm_features(img)
            features_list.append(glcm_feats)
        except Exception as e:
            print(f"Error saat mengekstrak fitur untuk gambar {i+1}: {e}")

    if not features_list:
        print("Tidak ada fitur yang diekstrak. Ini mungkin disebabkan oleh masalah dalam pemrosesan gambar atau semua gambar gagal.")
    else:
        X = np.array(features_list) # Matriks fitur
        y_str = np.array(labels_data) # Label dalam bentuk string

        # Encode label string ke integer karena model ML memerlukan input numerik
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y_str) # Label dalam bentuk angka

        print(f"Bentuk matriks fitur (X): {X.shape}")
        print(f"Bentuk label (y): {y.shape}")

        # Membagi dataset menjadi data latih dan data uji
        # Paper menyebutkan 60% pelatihan dan 30% pengujian.
        # Kita akan menggunakan test_size=0.3 untuk 30% data uji.
        # Stratify=y penting untuk menjaga proporsi kelas dalam data latih dan uji,
        # terutama jika jumlah sampel per kelas tidak seimbang.
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                test_size=0.30, # 30% untuk data uji, sisanya untuk data latih
                random_state=42, # Untuk reproduktifitas hasil
                stratify=y # Menjaga proporsi kelas
            )
        except ValueError as e:
             print(f"Error saat train_test_split (kemungkinan karena terlalu sedikit sampel per kelas untuk stratifikasi): {e}")
             print("Pertimbangkan untuk menggunakan dataset yang lebih besar atau menghapus stratifikasi jika kelas terlalu kecil.")
             # Fallback ke non-stratified jika terjadi kesalahan, meskipun tidak ideal
             X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.30, random_state=42
            )

        print(f"Jumlah sampel latih: {X_train.shape[0]}")
        print(f"Jumlah sampel uji: {X_test.shape[0]}")

Kelas ekspresi yang ditemukan: ['jijik', 'marah', 'sedih', 'senang', 'takut']

Proses pemuatan selesai.
Total gambar yang diproses: 783
Gambar yang berhasil dijajarkan (aligned) dengan dlib: 725
Gambar yang gagal dijajarkan (menggunakan gambar asli): 58
Berhasil memuat 783 gambar.
Mengekstrak fitur GLCM...

Bentuk matriks fitur (X): (783, 16)
Bentuk label (y): (783,)
Jumlah sampel latih: 548
Jumlah sampel uji: 235


In [27]:
print("Melatih classifier k-Nearest Neighbors...")
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

print("Mengevaluasi model...")
y_pred_train = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Akurasi Pelatihan: {train_accuracy * 100:.2f}%")

y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Akurasi Pengujian: {test_accuracy * 100:.2f}%")

print("\nLaporan Klasifikasi (Data Uji):")
report = classification_report(y_test, y_pred_test, target_names=label_encoder.classes_, zero_division=0)
print(report)

Melatih classifier k-Nearest Neighbors...
Mengevaluasi model...
Akurasi Pelatihan: 52.19%
Akurasi Pengujian: 27.66%

Laporan Klasifikasi (Data Uji):
              precision    recall  f1-score   support

       jijik       0.32      0.49      0.38        72
       marah       0.15      0.18      0.17        33
       sedih       0.20      0.12      0.15        34
      senang       0.25      0.10      0.14        31
       takut       0.31      0.26      0.29        65

    accuracy                           0.28       235
   macro avg       0.25      0.23      0.22       235
weighted avg       0.27      0.28      0.26       235

